In [1]:
import pandas as pd
from datetime import datetime
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pickle
from scipy.ndimage import label

In [2]:
HWs_lb_nov_mar = pd.read_csv('data/HWs_KJS_1996_2006_nov_mar.csv', sep='\t')
HWs_lb_nov_mar['date'] = pd.to_datetime(HWs_lb_nov_mar['date'])
HWs_lb_nov_mar

,date,temperature,p90_har,heatwave,label
0,1996-02-16,4.8,3.260798,True,1
1,1996-02-17,3.7,3.253920,True,1
2,1996-02-18,5.3,3.246030,True,1
3,1996-02-19,5.6,3.237139,True,1
4,1996-02-20,3.4,3.227258,True,1
...,...,...,...,...,...
122,2006-03-27,4.2,2.402470,True,80
123,2006-03-28,3.0,2.369998,True,80
124,2006-03-29,2.6,2.337291,True,80
125,2006-03-30,3.2,2.304368,True,80


In [3]:
filename = '/home/cr2/cmtorres/repos/ARs_SSI_FEL/data/ARs_Guan_AP_day.csv'
ARs_fall = pd.read_csv(filename, sep='\t', parse_dates=['date'])
ARs_fall

,date,ARs,date.1
0,1980-01-12,13.00005,1980-01-12 18:00:00
1,1980-01-15,6.00095,1980-01-15 00:00:00
2,1980-01-17,3.00014,1980-01-17 18:00:00
3,1980-01-18,10.00360,1980-01-18 18:00:00
4,1980-01-19,5.00356,1980-01-19 06:00:00
...,...,...,...
1582,2022-12-17,5.00072,2022-12-17 12:00:00
1583,2022-12-19,5.00174,2022-12-19 12:00:00
1584,2022-12-20,5.00513,2022-12-20 18:00:00
1585,2022-12-21,5.00267,2022-12-21 18:00:00


In [20]:
HWs_ARs_SSI = pd.merge(HWs_lb_nov_mar, ARs_fall, on='date', how='inner')

In [21]:
HWs_ARs_SSI

,date,temperature,p90_har,heatwave,label,ARs,date.1
0,1996-02-16,4.8,3.260798,True,1,3.00185,1996-02-16 18:00:00
1,1996-02-17,3.7,3.253920,True,1,4.00261,1996-02-17 00:00:00
2,1996-02-18,5.3,3.246030,True,1,5.00290,1996-02-18 18:00:00
3,1996-02-19,5.6,3.237139,True,1,5.00259,1996-02-19 12:00:00
4,1996-03-05,3.1,2.993946,True,2,13.00230,1996-03-05 06:00:00
...,...,...,...,...,...,...,...
72,2006-03-22,3.4,2.560661,True,79,7.00096,2006-03-22 18:00:00
73,2006-03-28,3.0,2.369998,True,80,6.00218,2006-03-28 18:00:00
74,2006-03-29,2.6,2.337291,True,80,4.00207,2006-03-29 00:00:00
75,2006-03-30,3.2,2.304368,True,80,9.00330,2006-03-30 18:00:00


In [22]:
filename = '/home/cr2/cmtorres/repos/ARs_SSI_FEL/data/KSJ_met_daily_1996_2020.csv'

df = pd.read_csv(filename, sep='\t')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df

,Timestamp,WS(m/s),WDv(deg),Ta(C),RH(%),Prs(hPa),Prcp(mm),Snow(cm),Rsw(W/m2)
0,1996-01-01,4.4,337.5,3.3,91.0,987.0,6.7,0.0,92.9
1,1996-01-02,8.9,312.4,1.5,91.4,985.8,4.5,0.0,113.0
2,1996-01-03,6.3,330.7,2.1,90.1,991.0,0.0,0.0,239.6
3,1996-01-04,6.7,4.1,4.1,88.3,991.1,0.0,0.0,256.1
4,1996-01-05,3.4,95.2,4.1,87.3,981.8,0.0,0.0,86.1
...,...,...,...,...,...,...,...,...,...
8852,2020-03-27,8.4,231.9,-3.9,72.6,972.5,0.1,5.0,88.3
8853,2020-03-28,7.6,290.1,-0.5,77.0,993.7,0.3,0.0,93.1
8854,2020-03-29,7.1,296.3,1.7,90.9,994.4,0.0,0.0,44.2
8855,2020-03-30,7.5,304.3,1.9,92.6,987.9,2.1,0.0,47.0


In [23]:
#df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # Asegura que sea datetime
df['month'] = df['Timestamp'].dt.month
df_nov_mar = df[df['month'].isin([11, 12, 1, 2, 3])].copy()
df_nov_mar.drop(columns='month', inplace=True)  # Limpia la columna auxiliar
var = ['Timestamp', ' WS(m/s)', ' Prcp(mm)']
df_nov_mar = df_nov_mar[var].rename(columns={'Timestamp':'date', ' WS(m/s)':'ws', ' Prcp(mm)':'rainfall'})


In [46]:
HWs_ARs_MET_SSI = pd.merge(HWs_ARs_SSI, df_nov_mar, on='date', how='left')
p99 = df_nov_mar[['ws', 'rainfall']].quantile(0.90)
HWs_ARs_MET_SSI

,date,temperature,p90_har,heatwave,label,ARs,date.1,ws,rainfall
0,1996-02-16,4.8,3.260798,True,1,3.00185,1996-02-16 18:00:00,12.1,22.0
1,1996-02-17,3.7,3.253920,True,1,4.00261,1996-02-17 00:00:00,9.9,2.2
2,1996-02-18,5.3,3.246030,True,1,5.00290,1996-02-18 18:00:00,9.6,9.7
3,1996-02-19,5.6,3.237139,True,1,5.00259,1996-02-19 12:00:00,9.8,4.2
4,1996-03-05,3.1,2.993946,True,2,13.00230,1996-03-05 06:00:00,6.1,4.4
...,...,...,...,...,...,...,...,...,...
72,2006-03-22,3.4,2.560661,True,79,7.00096,2006-03-22 18:00:00,11.0,0.0
73,2006-03-28,3.0,2.369998,True,80,6.00218,2006-03-28 18:00:00,11.9,3.9
74,2006-03-29,2.6,2.337291,True,80,4.00207,2006-03-29 00:00:00,10.8,0.4
75,2006-03-30,3.2,2.304368,True,80,9.00330,2006-03-30 18:00:00,12.8,1.0


In [47]:
# Filtrar com base nas colunas 'ws' e 'rainfall' usando os thresholds de p99
HWs_ARs_MET_SSI_extreme = HWs_ARs_MET_SSI[
    (HWs_ARs_MET_SSI['rainfall'] >= p99['rainfall'])
]

In [48]:
HWs_ARs_MET_SSI_extreme

,date,temperature,p90_har,heatwave,label,ARs,date.1,ws,rainfall
0,1996-02-16,4.8,3.260798,True,1,3.00185,1996-02-16 18:00:00,12.1,22.0
2,1996-02-18,5.3,3.246030,True,1,5.00290,1996-02-18 18:00:00,9.6,9.7
6,1996-03-07,4.8,2.947861,True,2,13.00014,1996-03-07 18:00:00,8.9,5.5
7,1996-03-19,3.0,2.621616,True,3,14.00190,1996-03-19 18:00:00,8.8,10.8
12,1996-12-22,3.4,2.223737,True,13,6.00156,1996-12-22 18:00:00,9.6,16.4
15,1997-01-10,3.8,2.804588,True,14,3.00194,1997-01-10 06:00:00,12.1,5.6
29,1998-11-11,1.1,0.936654,True,31,6.00200,1998-11-11 12:00:00,12.1,5.4
36,1999-11-14,1.6,1.010791,True,41,6.00251,1999-11-14 00:00:00,5.6,7.7
40,1999-11-19,1.7,1.140502,True,42,5.00162,1999-11-19 18:00:00,13.2,5.8
44,2000-02-03,3.5,3.254920,True,44,6.00217,2000-02-03 18:00:00,17.1,27.9
